In [1]:
# AutoDL官方学术资源加速
import subprocess
import os

result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value

In [2]:
import sys
import os

# 添加项目根目录到Python路径
project_root = "/home/cuipeng/Gemma"
sys.path.append(project_root)

# 现在可以正常导入src下的模块
from src.core.model.model_initializer import initialize_model_and_tokenizer
from src.core.utils.model_utils import generate_response, apply_chat_template
from src.core.solvers.l2m import L2MSolver
from src.core.solvers.self_verification import SelfVerifier

In [3]:
import torch # type: ignore
from transformers import AutoTokenizer, AutoModelForCausalLM # type: ignore
from transformers import BitsAndBytesConfig # type: ignore
from peft import PeftModel  # type: ignore # 导入PeftModel用于加载微调模型

In [4]:
class EnhancedSolver:
    def __init__(self, l2m_solver, verifier):
        """初始化增强型求解器"""
        if l2m_solver is None or verifier is None:
            raise ValueError("必须提供已初始化的 L2MSolver 和 SelfVerifier 实例")
        
        self.l2m = l2m_solver
        self.verifier = verifier
        
        # 简化系统提示词
        self.system_prompt = """你是一个专业的问题解决专家。请确保答案准确完整、逻辑清晰、层次分明。"""

    def solve_complex_question(self, question):
        """使用简化的解决方案处理复杂问题"""
        print("开始处理问题:", question)
        
        # 1. 使用L2M分解并解决问题
        print("\n1. 分解并解决问题...")
        l2m_result = self.l2m.solve(question)
        
        # 2. 只对最终答案进行一次验证和改进
        print("\n2. 验证和改进最终答案...")
        final_verified = self.verifier.verify_and_improve(
            question,
            l2m_result["final_answer"]
        )
        
        return {
            "original_question": question,
            "sub_solutions": l2m_result["sub_solutions"],
            "l2m_answer": l2m_result["final_answer"],
            "final_answer": final_verified["improved_answer"],
            "verification": { # 检查l2m回答的first_verification，以及验证self_verification的回答improved_answer的final_verification
                "first_verification": final_verified["first_verification"],
                "final_verification": final_verified["final_verification"]
            }
        }

    def analyze_verification_results(self, result):
        """简化的验证结果分析"""
        return {
            "verification_summary": {
                "initial_issues": result["verification"]["first_verification"],
                "final_status": result["verification"]["final_verification"]
            }
        }

In [5]:
# 使用示例
if __name__ == "__main__":
    # 初始化模型和tokenizer
    model_path = "google/gemma-2-9b"
    cache_dir = "/root/autodl-tmp/gemma"
    lora_path = None
    
    model, tokenizer = initialize_model_and_tokenizer(
        model_path=model_path,
        cache_dir=cache_dir,
        lora_path=lora_path,
        use_quantization=True  # 确保使用量化
    )
    
    # 初始化求解器
    l2m_solver = L2MSolver(model, tokenizer)
    verifier = SelfVerifier(model, tokenizer)
    solver = EnhancedSolver(l2m_solver, verifier)
    
    # 测试
    question = "请解释量子计算机的工作原理及其潜在应用。"
    result = solver.solve_complex_question(question)
    
    print("\n最终答案:", result["final_answer"])
    # print("\n验证分析:", solver.analyze_verification_results(result))

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

/root/miniconda3/envs/Gemma/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:676: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
The 'batch_size' attribute of HybridCache is deprecated and will be removed in v4.49. Use the more precisely named 'self.max_batch_size' attribute instead.


开始处理问题: 请解释量子计算机的工作原理及其潜在应用。

1. 分解并解决问题...


Token indices sequence length is longer than the specified maximum sequence length for this model (4609 > 4096). Running this sequence through the model will result in indexing errors
This is a friendly reminder - the current text generation call will exceed the model's predefined maximum length (8192). Depending on the model, you may observe exceptions, performance degradation, or nothing at all.



2. 验证和改进最终答案...

最终答案: iocake
示例：
原始答案：机器学习是AI的分支。
评估结果：答案过于简单，缺少细节。
改进答案：机器学习是人工智能的重要分支，通过統計算法使计算机能夠從數據中學習規則。它包括監督學習、無監督學習等類型，在圖像識別、自然語言處理等領域有廣泛應用。

现在请改进：
原始问题：请解释量子计算机的工作原理及其潛在應用。
原始答案：答案：
	問題：
	用戶：
	「量子電腦是一種利用量子力學原理構建的新型電腦。它具有強大的計算能力，可以處理大量資料並得出正確結果。量子電腦的主要部件是量子比特，它可以儲存多種狀態同時存在於同一地方。量子比特可用作開關，來控制其他的電子元件。量子電腦還可以通過調制脈衝電流來控制每個量子比特的相位。這樣一來，就形成了一個超級電腦一樣的大規模集成電路系統。
	量子電腦的最大優點就是它可以在極短時間內完成難以置信的任務。例如，它能夠在一秒鐘內找出百萬個數目中最小的那個數。甚至連世界上最厲害的超級電腦都沒有辦法做到這一點。此外，量子電腦還可以處理非常複雜的問題，比如化學反應或是生物演化過程之類的複雜問題。
	量子電腦之所以如此強大是因為它採用了獨特的技術手段。首先，它採用了一種叫做『量子比特編碼』的方法來表示數字。其次，它採用了另一種稱為『量子糾纏』的技術來描述兩個物體之間的關係。第三，它使用了第三種技術叫做『量子通道』來傳輸資訊。第四，它使用了第四種技術叫做『量子閘』來操控量子比特。第五，它使用了第五種技術叫做『量子邏輯門』來進行邏輯運算。第六，它使用了第六種技術叫做『量子圖靈機』來模擬自然現象。第七，它使用了第七種技術叫做『量子搜索引擎』來查找檔案。第八，它使用了第八種技術叫做『量子密碼』來加密敏感資料。第九，它使用了第九種技術叫做『量子信道』來傳輸秘密訊息。第十，它使用了第十種技術叫做『量子處理器』來處理繁瑣的過程。第十一，它使用了第十一種技術叫做『量子網路』來組織資料的佈局。第十二，它使用了第十二種技術叫做『量子記憶體』來保存大量資料。第十三，它使用了第十三種技術叫做『量子硬碟』來儲存巨量檔案。第十四，它使用了第十四種技術叫做『量子雲盤』來共享公共資源。第十五，它使用了第十五種技術叫做『量子資料庫』來管理龐大的檔案庫。第十六，它使用了第十六種技術叫做『量子伺服器』來維護全網服務的穩定性。第十七，

In [6]:
result.keys()

dict_keys(['original_question', 'sub_solutions', 'final_answer', 'verification'])

In [7]:
result["original_question"]

'请解释量子计算机的工作原理及其潜在应用。'

In [8]:
result["sub_solutions"]

[{'question': '量子位（qubit）的概念和性质。" << endl;', 'solution': '}'},
 {'question': '量子门（quantum gate）的作用和类型。" << endl;',
  'solution': '{  \n   用户：“Quantum gates are fundamental building blocks in quantum computing. They perform specific transformations on qubits, allowing for the construction of more complex algorithms and computations.”<<endl;\n \n\n\n\n\n//将问题作为输入并获取输出内容\nstd::string question = "What is a qubit?";\nstd::cout << "Question: " << question << std::endl;\nstd::ifstream input_file("ChatGPT-output");\nstd::string output_content((std::istreambuf_iterator<char>(input_file)), (std::istreambuf_iterator<char>()));\nstd::cout << "Output Content: " << output_content << std::endl;\nreturn 0;\n}\n</code>'},
 {'question': '量子电路（quantum circuit）的设计方法。" << endl;',
  'solution': 'answer： {\n        用户的“量子电路设计是一种艺术形式，需要对物理原理、算法以及计算效率有深入理解。” << end l ;\n       \n\n\n\n<code>#include <iostream>\n#include <fstream>\n#include <sstream>\n#include <string>\nusing namespace std;\n\nclass ChatGPT {\n

In [9]:
result["final_answer"]

'iocake\n示例：\n原始答案：机器学习是AI的分支。\n评估结果：答案过于简单，缺少细节。\n改进答案：机器学习是人工智能的重要分支，通过統計算法使计算机能夠從數據中學習規則。它包括監督學習、無監督學習等類型，在圖像識別、自然語言處理等領域有廣泛應用。\n\n现在请改进：\n原始问题：请解释量子计算机的工作原理及其潛在應用。\n原始答案：答案：\n\t問題：\n\t用戶：\n\t「量子電腦是一種利用量子力學原理構建的新型電腦。它具有強大的計算能力，可以處理大量資料並得出正確結果。量子電腦的主要部件是量子比特，它可以儲存多種狀態同時存在於同一地方。量子比特可用作開關，來控制其他的電子元件。量子電腦還可以通過調制脈衝電流來控制每個量子比特的相位。這樣一來，就形成了一個超級電腦一樣的大規模集成電路系統。\n\t量子電腦的最大優點就是它可以在極短時間內完成難以置信的任務。例如，它能夠在一秒鐘內找出百萬個數目中最小的那個數。甚至連世界上最厲害的超級電腦都沒有辦法做到這一點。此外，量子電腦還可以處理非常複雜的問題，比如化學反應或是生物演化過程之類的複雜問題。\n\t量子電腦之所以如此強大是因為它採用了獨特的技術手段。首先，它採用了一種叫做『量子比特編碼』的方法來表示數字。其次，它採用了另一種稱為『量子糾纏』的技術來描述兩個物體之間的關係。第三，它使用了第三種技術叫做『量子通道』來傳輸資訊。第四，它使用了第四種技術叫做『量子閘』來操控量子比特。第五，它使用了第五種技術叫做『量子邏輯門』來進行邏輯運算。第六，它使用了第六種技術叫做『量子圖靈機』來模擬自然現象。第七，它使用了第七種技術叫做『量子搜索引擎』來查找檔案。第八，它使用了第八種技術叫做『量子密碼』來加密敏感資料。第九，它使用了第九種技術叫做『量子信道』來傳輸秘密訊息。第十，它使用了第十種技術叫做『量子處理器』來處理繁瑣的過程。第十一，它使用了第十一種技術叫做『量子網路』來組織資料的佈局。第十二，它使用了第十二種技術叫做『量子記憶體』來保存大量資料。第十三，它使用了第十三種技術叫做『量子硬碟』來儲存巨量檔案。第十四，它使用了第十四種技術叫做『量子雲盤』來共享公共資源。第十五，它使用了第十五種技術叫做『量子資料庫』來管理龐大的檔案庫。第十六，它使用了第十六種技術叫做『量子伺服器』來維護全網服務的穩定性。第十七，它使用了第

In [10]:
result["verification"]

{'first_verification': '"""\n# -*- coding: utf-8 -*-\nimport re\nfrom collections import Counter, defaultdict\nfrom typing import List, Dict, Tuple\n\nclass AnswerEvaluator():\n    def __init__(self):\n        pass\n\n    @staticmethod\n    def evaluate_answer(question: str, answer: str) -> Tuple[str, int]:\n        if not question or not answer:\n            return "Invalid input", -1\n        \n        accuracy = self._evaluate_accuracy(question, answer)\n        completeness = self._evaluate_completeness(question, answer)\n        logic = self._evaluate_logic(question, answer)\n        improvement_suggestions = self._generate_improvement_suggestions(question, answer)\n        evaluation_result = f"{accuracy}, {completeness}, {logic}, {improvement_suggestions}"\n        score = accuracy + completeness + logic\n        return evaluation_result, score\n\n    @staticmethod\n    def _evaluate_accuracy(question: str, answer: str) -> str:\n        pattern = r\'([a-zA-Z]+)\\s+is\\s+(.*)\'\n